In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.preprocess import preprocess

In [3]:
preprocess(
    "data/genome/gff_file.gff",
    "data/genome/fasta_file.fsa",
    "data/embeddings",
    "data/waern_2013",
    "data/samples.json",
    "data/processed",
    500,
)

100%|██████████| 17/17 [02:46<00:00,  9.79s/it]
